# Basic simplest version
In this file, using tf-idf with SVM abd Naive Bayes.  Comparing 50/50 80/20 90/10 data splits.

Working with nltk version 3.2.5 and scikit-learn version 0.20.1.

In [1]:
import nltk
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split

verbose = False


Splitting the training.csv into 80/20 split for training and validation.  Further splitting the training set int 80/20 where the 20 will be used to help tune the model. X is the training set, and y is the validation set.  X is then split further into X_train and X_test where X_test will be used for tuning.  


In [2]:
currentPath = os.path.join(os.getcwd(), "data", "")
train = pd.read_csv(currentPath + "train.csv")
print(train.head)

#split train
X, y = train_test_split(train, test_size = .20)

if(verbose):
    print( X)
    print("\n\n y: ", y)

X_train, X_test = train_test_split(
    X, test_size = .20)

if(verbose):
    print("X_train: ", X_train)
    print("\n\nX_test ", X_test)

<bound method NDFrame.head of             id                                               text author
0      id26305  This process, however, afforded me no means of...    EAP
1      id17569  It never once occurred to me that the fumbling...    HPL
2      id11008  In his left hand was a gold snuff box, from wh...    EAP
3      id27763  How lovely is spring As we looked from Windsor...    MWS
4      id12958  Finding nothing else, not even gold, the Super...    HPL
5      id22965  A youth passed in solitude, my best years spen...    MWS
6      id09674  The astronomer, perhaps, at this point, took r...    EAP
7      id13515        The surcingle hung in ribands from my body.    EAP
8      id19322  I knew that you could not say to yourself 'ste...    EAP
9      id00912  I confess that neither the structure of langua...    MWS
10     id16737  He shall find that I can feel my injuries; he ...    MWS
11     id16607  Here we barricaded ourselves, and, for the pre...    EAP
12     id19764  Herbe

Checking the sets:

In [3]:
if(verbose):
    print("X_train shape: ", X_train.shape)
    print("X_test shape: ", X_test.shape)
    print("y shape: ", y.shape)
    X_train.head(5)

Extracting features, using bag of words model. 

In [4]:
# for feature extraction
from sklearn.feature_extraction.text import CountVectorizer
# tf-idf
from sklearn.feature_extraction.text import TfidfTransformer
# SVM
from sklearn.linear_model import SGDClassifier
# NB
from sklearn.naive_bayes import MultinomialNB
# pipeline
from sklearn.pipeline import Pipeline

### Support Vector Machine

In [5]:
# step by step reference, no pipeline

# count words
count_vect = CountVectorizer()
X_train_count = count_vect.fit_transform(X_train.text)
X_train_count.shape

# tf-idf
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_count)
X_train_tfidf.shape

# NB
clf = MultinomialNB().fit(X_train_tfidf, X_train.author)

## Pipeline
Using pipelines to help speed things up

In [6]:
svm_clf = Pipeline([("vect", CountVectorizer()),
                        ("tfidf", TfidfTransformer()),
                        ("clf-svm", SGDClassifier(loss = "hinge", penalty = "l2",
                                                tol = 1e-3, 
                                                random_state = 42)),])


### Multinomial NB

In [7]:
nb_clf = Pipeline([("vect", CountVectorizer()),
                    ("tfidf", TfidfTransformer()),
                    ("clf- nb", MultinomialNB(),)])


## Automate pipelines
Create lists and a dictionary to help iterate through the fit and predict parts. 

In [8]:
pipellines = [svm_clf, nb_clf]
pipe_dict = {0: "Support Vector Machine", 1: "Naive Bayes"}

## Reference
Check nltk and sklearn versions: https://stackoverflow.com/questions/28501072/how-to-check-which-version-of-nltk-scikit-learn-installed
